# Exploratory Data Analysis (EDA)

Dataset: `/Users/eb2007/Library/CloudStorage/OneDrive-UniversityofCambridge/Documents/PhD/data/data_c4_raw.csv` (≈700K rows)
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

In [ ]:
# Path to the large CSV file
csv_path = '/Users/eb2007/Library/CloudStorage/OneDrive-UniversityofCambridge/Documents/PhD/data/data_c4_raw.csv'

# Read a small sample to infer dtypes and preview
sample = pd.read_csv(csv_path, nrows=5000)
display(sample.info())
display(sample.head())

In [ ]:
# Try to load the full dataset
try:
    df = pd.read_csv(csv_path)
    print('Shape:', df.shape)
    print('Columns:', df.columns.tolist())
    print('Memory usage (MB):', df.memory_usage(deep=True).sum() / 1e6)
    display(df.head())
    display(df.tail())
    print('Missing values per column:')
    print(df.isnull().sum())
    display(df.describe(include="all"))
except MemoryError:
    print('MemoryError: Consider loading with chunksize or specifying dtypes.')
    df = None